In [1]:

import requests
import os
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from base64 import b64encode
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import selenium.webdriver.support.ui as ui

In [47]:

#siteUrl = 'https://www.indeed.com/q-data-scientist-l-Westford,-MA-jobs.html'
base_url = 'https://www.indeed.com'
site_url = base_url + '/jobs'
site_url = "?".join([site_url, "q=data+scientist"])
site_url = "&".join([site_url, "l=Westford%2C+MA"])
print(site_url)

https://www.indeed.com/jobs?q=data+scientist&l=Westford%2C+MA


In [48]:

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=site_url)
sitePageSoup = BeautifulSoup(sitePage.content, 'lxml')

In [56]:

# https://www.indeed.com/cmp/Hollingsworth-&-Vose-Company/jobs/Technician-c53010d10ff84f5c?
# sjdu=Zzi_VW2ygsY1fzh3Ma9ZsE4zIT1NTXCwgFBhdjeTC3Nm09-frVhymnozbr5xpJs_ZE6OOHgkTH1Df0IF5fBpzA

# https://www.indeed.com/viewjob?jk=052757188b817895&q=data+scientist&l=Westford%2C+MA&tk=1c0734nqaav53fsh&from=web
space_regex = re.compile('\s+')
row_div_list = sitePageSoup.find_all(name='div', class_=['row', 'result'])
reqs_regex = re.compile('ducat|xperience|equire')
for row_div in row_div_list:
    a_list = row_div.find_all(name='a')
    print(space_regex.sub(' ', a_list[0]['title'].strip()))
    job_url = base_url + a_list[0]['href']
    job_page = requests.get(url=job_url)
    job_soup = BeautifulSoup(job_page.content, 'lxml')
    job_text = job_soup.text
    text_middle = int(len(job_text)/2)
    for match in reqs_regex.finditer(string=job_text):
        text_start = match.start()
        print(job_text[text_start-50:text_start+50])
    print('-'*100)
    print()

Head- Data Management and Analytics
rove patient outcomes by generating the evidence required by healthcare decision makers to support p
ess and use of sanofi medicines and the insights required by Sanofi internal teams to develop and co
ces Strategies and Plans generating the insights required by healthcare decision makers to support p
ess and use of sanofi medicines and the insights required by Sanofi internal teams to develop and co
actful medicines</p><p><p>These responsibilities require an extensive experience in data management,
p><p>These responsibilities require an extensive experience in data management, visual analytics and
li>Delivers data management and visual analytics requirement on time on quality and on budget</li><l
BACKGROUND AND JOB-RELATED EXPERIENCE:</b><br><b>Education:</b></p></li></ul><ul><li>Bachelors’ degr
ial drug development processes</li></ul><p><p><b>Experience:</b></p><ul><li>5 plus years in healthca
are analytics/informatics and report development experi

In [ ]:

<div class="row  result" id="pj_c53010d10ff84f5c" data-jk="c53010d10ff84f5c" data-advn="6354767581944530">
            <!-- Previously this variable was used to indicate job board jobs, we have replaced that with a more accurate 
    source type check -->
            <a target="_blank" id="sja1" data-tn-element="jobTitle" class="jobtitle turnstileLink" 
        href="https://www.indeed.com/pagead/clk?mo=r&amp;ad=-...-...-...-...-...-...=&amp;
        p=1&amp;sk=&amp;fvj=1&amp;tk=1c0734nqaav53fsh&amp;jsa=410" title="Technician" rel="noopener nofollow" 
        onmousedown="sjomd('sja1'); clk('sja1');" onclick="setRefineByCookie([]); sjoc('sja1',0); convCtr('SJ')"
        >Technician</a>

            <br>
            <div class="sjcl">
            <span class="company">
    Hollingsworth &amp; Vose Company</span>

 - <span class="location">West Groton, MA</span>
            </div>
            <div class="">
                <table cellspacing="0" cellpadding="0" border="0"><tbody><tr><td class="snip">
                        <span class="summary">
                            Effectively communicate results to <b>Scientists</b> and others describing the results 
                    obtained as well as other observations using <b>data</b> tables, graphs, and small...</span>
                    </td></tr></tbody></table>
                </div>

                <div class="sjCapt">
                    <div class="iaP">
    <span class="iaLabel"> Easily apply</span>
</div>
<div class="result-link-bar-container">
                            <div class="result-link-bar"><span class=" sponsoredGray ">Sponsored</span> - 
    <span class="date">1 day ago</span> <span id="tt_set_10" class="tt_set">  -  
    <a id="sj_c53010d10ff84f5c" href="#" class="sl resultLink save-job-link " 
    onclick="changeJobState('c53010d10ff84f5c', 'save', 'linkbar', true, ''); return false;" 
    title="Save this job to my.indeed">save job</a></span><div id="editsaved2_c53010d10ff84f5c" 
    class="edit_note_content" style="display:none;"></div>
    <script>if (!window['sj_result_c53010d10ff84f5c']) {window['sj_result_c53010d10ff84f5c'] = {};
                                                       }window['sj_result_c53010d10ff84f5c']['showSource'] = false; 
    window['sj_result_c53010d10ff84f5c']['source'] = "Indeed"; window['sj_result_c53010d10ff84f5c']['loggedIn'] = false; 
    window['sj_result_c53010d10ff84f5c']['showMyJobsLinks'] = false;
    window['sj_result_c53010d10ff84f5c']['undoAction'] = "unsave";
    window['sj_result_c53010d10ff84f5c']['relativeJobAge'] = "1 day ago";
    window['sj_result_c53010d10ff84f5c']['jobKey'] = "c53010d10ff84f5c"; 
    window['sj_result_c53010d10ff84f5c']['myIndeedAvailable'] = true;
    window['sj_result_c53010d10ff84f5c']['showMoreActionsLink'] = false; 
    window['sj_result_c53010d10ff84f5c']['resultNumber'] = 10; 
    window['sj_result_c53010d10ff84f5c']['jobStateChangedToSaved'] = false; 
    window['sj_result_c53010d10ff84f5c']['searchState'] = "q=data scientist&amp;l=Westford%2C+MA"; 
    window['sj_result_c53010d10ff84f5c']['basicPermaLink'] = "https://www.indeed.com"; 
    window['sj_result_c53010d10ff84f5c']['saveJobFailed'] = false; 
    window['sj_result_c53010d10ff84f5c']['removeJobFailed'] = false; 
    window['sj_result_c53010d10ff84f5c']['requestPending'] = false; 
    window['sj_result_c53010d10ff84f5c']['notesEnabled'] = false; window['sj_result_c53010d10ff84f5c']['currentPage'] = "serp"; window['sj_result_c53010d10ff84f5c']['sponsored'] = true;window['sj_result_c53010d10ff84f5c']['showSponsor'] = true;window['sj_result_c53010d10ff84f5c']['reportJobButtonEnabled'] = false; window['sj_result_c53010d10ff84f5c']['showMyJobsHired'] = false; window['sj_result_c53010d10ff84f5c']['showSaveForSponsored'] = true; window['sj_result_c53010d10ff84f5c']['showJobAge'] = true;</script></div></div>
                        <div class="tab-container">
                            <div class="sign-in-container result-tab"></div>
                            <div class="tellafriend-container result-tab email_job_content"></div>
                        </div>
                    </div>
            </div>

In [34]:
    if len(a_list)==0:
        print(row_div)
        print()
        print('-'*100)
        print()

In [20]:

def has_class_and_id(tag):
    
    return tag.has_attr('class') and tag.has_attr('id')

# <span class="summary">We need a <b>data</b> <b>scientist</b> and <b>data</b> wrangler. Maybe survey <b>data</b>. 
# Better still if you have some demonstrable experience with more advanced machine learning methods...</span>
summary_list = sitePageSoup.find_all(name='span', class_='summary')
for summary in summary_list:
    print(summary.text.strip())
    print()

Accountabilities Lead the recruitment and allocation of appropriately skilled data scientists/health informatics to projects to ensure delivery of value...

Degree (in Engineering, Computer Science, Mathematics, or other scientific field of study) and 0-1 years of relevant experience, a Master’s degree (scientific...

He/she will be expected to lead experimental design, interpret data, troubleshoot experiments, coordinate junior scientists, and present work at both internal...

Integration of business data into automation workflows and scientific data into. Familiarity with scientific modeling and data preferred....

Knowledge of data science, data mining, machine learning and statistics. Knowledge of R and R plotting and/or Python matplotlib, Scikit, Matlab....

We are passionate about deep learning, reinforcement learning, semi-supervised and unsupervised learning, but are also considering other areas of machine...

Research Scientist - Computer Vision / Machine Learning. And depth i

In [3]:

# <a target="_blank" id="sja5" data-tn-element="jobTitle" class="jobtitle turnstileLink" 
# href="https://www.indeed.com/pagead/clk?mo=r&amp;ad=-...-...-...-...-...-...-...&
# amp;p=5&amp;sk=&amp;fvj=1&amp;tk=1c06s8995av53coj&amp;jsa=6565" 
# title="Perception Scientist for Marine Autonomy" rel="noopener nofollow" 
# onmousedown="sjomd('sja5'); clk('sja5');" onclick="setRefineByCookie([]); sjoc('sja5',0); convCtr('SJ')"
# >Perception <b>Scientist</b> for Marine Autonomy</a>
siteCss = '#sja5'
siteLinks = sitePageSoup.select(siteCss)
print(siteLinks)

max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[0]["href"].split('/')[-1].split('-')[1].split(',')[0])
print(max_page)

WebDriverException: Message: connection refused


In [6]:
#<h1 class="srp-header">25,589 Used Vehicles for sale</h1>

for parser in ['lxml', 'html5lib', 'html.parser']:
    sitePageSoup = BeautifulSoup(page_html, parser)
    print(parser, len(sitePageSoup.select('h1.srp-header')))

#max_page = int(sitePageSoup.find_all("a", class_="js-last-page")[0].text)
#print(max_page)

lxml 1
html5lib 0
html.parser 1


In [10]:
sitePageSoup = BeautifulSoup(page_html, 'lxml')
srp_header = sitePageSoup.select('h1.srp-header')
if len(srp_header):
    max_record_string = srp_header[0].get_text().strip()
    print(max_record_string)
    max_record = int(re.sub('\\D+', '', max_record_string))
    print(max_record)
    max_page = round(max_record/50)
    print(max_page)

25,592 Used Vehicles for sale
25592
512


In [25]:
from random import randint
import urllib.parse

i = randint(1, max_page)
tagPageUrl = siteUrl+"&page="+str(i)
print(tagPageUrl)

# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl, proxies=proxies)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

links = tagPageSoup.find_all("a", class_="listing-row__photo")
link = links[randint(0,len(links)-1)]
url = urllib.parse.urljoin(tagPageUrl, re.sub(' ', '%20', link['href']))
print(url)

https://www.cars.com/for-sale/searchresults.action/?perPage=50&rd=99999&searchSource=UTILITY&sf1Dir=DESC&sf1Nm=price&sf2Dir=ASC&sf2Nm=miles&slrTypeId=28879&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNewLowPrice&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&sortFeatures=buryUsedLowPrice&stkTypId=28881&yrId=20141&yrId=20142&yrId=20143&yrId=20144&yrId=20145&yrId=20180&yrId=20197&yrId=20198&yrId=20199&yrId=20200&yrId=20201&yrId=27381&yrId=34923&yrId=39723&yrId=47272&yrId=51683&yrId=56007&yrId=58487&zc=60606&page=31
https://www.cars.com/vehicledetail/detail/692753718/overview/


In [24]:
import re

page = requests.get(url=url, proxies=proxies)
soup = BeautifulSoup(page.content, 'lxml')

dealer_css = 'h4.dealer-name.cui-delta > span'
dealer_link_text = "Private Seller"
dealer_link = soup.select(dealer_css)
is_dealer = False
if len(dealer_link):
    match = re.search(dealer_link_text, dealer_link[0].get_text().strip())
    if match:
        is_dealer = False
    else:
        is_dealer = True
else:
    is_dealer = True
if not is_dealer:

    model = soup.find_all("h1", class_="vdp-header__title")
    if(len(model)):
        model = model[0].get_text().strip()
    else:
        model = "Unknown"
    print(model)

    price = soup.find_all("strong", class_='vdp-header__price--primary')
    if(len(price)):
        price = price[0].get_text().strip()
    else:
        price = "Unknown"
    print(price)

    phone_css = 'span.phone > span > span'
    phone = soup.select(phone_css)
    if(len(phone)):
        phone = phone[0].get_text().strip()
    else:
        phone = ""
    #print(phone)
    if(len(phone)):
        bigFile = open('cars.txt', 'a')
        bigFile.write(model + '\t' + price + '\t' + phone + '\n')
        bigFile.close()

Private Seller
Used 2008 Ford F250 Lariat Crew Cab Super Duty
$39,000
(908) 878-3257


In [6]:
for i in range(1, max_page+1):
    tagPageUrl = siteUrl+"&page="+str(i)

    # Retrieve the page with tag results and set it up to be scraped
    tagPage = requests.get(url=tagPageUrl, proxies=proxies)
    tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

    links = tagPageSoup.find_all("a", class_="listing-row__photo")
    for link in links:
        url = urllib.parse.urljoin(tagPageUrl, re.sub(' ', '%20', link['href']))
        page = requests.get(url=url, proxies=proxies)
        soup = BeautifulSoup(page.content, 'lxml')
        
        dealer_css = 'h4.dealer-name.cui-delta > span'
        dealer_link_text = "Private Seller"
        dealer_link = soup.select(dealer_css)
        is_dealer = False
        if len(dealer_link):
            match = re.search(dealer_link_text, dealer_link[0].get_text().strip())
            if match:
                is_dealer = False
            else:
                is_dealer = True
        else:
            is_dealer = True
        if not is_dealer:

            model = soup.find_all("h1", class_="vdp-header__title")
            if(len(model)):
                model = model[0].get_text().strip()
            else:
                model = "Unknown"
            #print(model)

            price = soup.find_all("strong", class_='vdp-header__price--primary')
            if(len(price)):
                price = price[0].get_text().strip()
            else:
                price = "Unknown"
            #print(price)

            phone_css = 'span.phone > span > span'
            phone = soup.select(phone_css)
            if(len(phone)):
                phone = phone[0].get_text().strip()
            else:
                phone = ""
            #print(phone)
            if(len(phone)):
                bigFile = open('cars.txt', 'a')
                bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                bigFile.close()

In [3]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl, proxies=proxies)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

In [25]:
html_doc = """
<cars-shop-srp-pagination><cui-pagination page="page" per-page="perPage" number-of-pages="numberOfPages" results-per-page-options="resultsPerPageOptions"><cui-page-button><a class="button previous-page" ng-click="prevPage($event)" ng-disabled="page === 1" rel="prev" ng-attr-data-linkname="{{dtmLinkPrev}}" data-linkname="" disabled="disabled">Prev</a> <a class="button next-page" ng-click="nextPage($event)" ng-disabled="page === numberOfPages" rel="next" ng-attr-data-linkname="{{dtmLinkNext}}" data-linkname="">Next</a> </cui-page-button> <cui-page-list><ul class="page-list"> <!----> <!----> <!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-1 current-page" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">1</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-2" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">2</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-3" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">3</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-4" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">4</a> </li><!----><li ng-repeat="p in pageList track by $index"> <a class="js-page-5" ng-click="goToPage($event, p)" ng-class="{'current-page': p === page}" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" data-linkname="">5</a> </li><!----> <!----><li ng-if="showLast">...</li><!----> <!----><li ng-if="showLast"> <a class="js-last-page" ng-click="goToPage($event, numberOfPages)">50</a> </li><!----> </ul> </cui-page-list> <cui-page-result-count><div class="select rpp"> <select ng-model="rppSelectModel.perPage" ng-model-options="{updateOn: 'default change', debounce: {default: 250, change: 0}}" ng-change="rppChange()" ng-options="rpp.text for rpp in rppSelectModel.options track by rpp.value" ng-attr-data-linkname="{{:: dtmListing}}" class="ng-pristine ng-untouched ng-valid ng-not-empty" data-linkname=""><option label="10 Per Page" value="10">10 Per Page</option><option label="20 Per Page" value="20">20 Per Page</option><option label="30 Per Page" value="30">30 Per Page</option><option label="50 Per Page" value="50" selected="selected">50 Per Page</option><option label="100 Per Page" value="100">100 Per Page</option></select> </div> </cui-page-result-count> </cui-pagination> </cars-shop-srp-pagination>
"""

soup2 = BeautifulSoup(html_doc, 'html.parser')

print(soup2.prettify())

<cars-shop-srp-pagination>
 <cui-pagination number-of-pages="numberOfPages" page="page" per-page="perPage" results-per-page-options="resultsPerPageOptions">
  <cui-page-button>
   <a class="button previous-page" data-linkname="" disabled="disabled" ng-attr-data-linkname="{{dtmLinkPrev}}" ng-click="prevPage($event)" ng-disabled="page === 1" rel="prev">
    Prev
   </a>
   <a class="button next-page" data-linkname="" ng-attr-data-linkname="{{dtmLinkNext}}" ng-click="nextPage($event)" ng-disabled="page === numberOfPages" rel="next">
    Next
   </a>
  </cui-page-button>
  <cui-page-list>
   <ul class="page-list">
    <!-- -->
    <!-- -->
    <!-- -->
    <li ng-repeat="p in pageList track by $index">
     <a class="js-page-1 current-page" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">
      1
     </a>
    </li>
    <!-- -->
    <li ng-repeat="p in pageList track by $index">

In [26]:
match = re.search(r'<a[^>]+class="js-last-page"[^>]+>([^><]+)</a>', str(soup2))
if match:
    max_page_string = match.group(1)
else:
    max_page_string = ""
if len(max_page_string):
    print(max_page_string)
    max_page = int(re.sub('\D+', '', max_page_string))
    print(max_page)

50
50


In [34]:
soup2.select("a")

[<a class="button previous-page" data-linkname="" disabled="disabled" ng-attr-data-linkname="{{dtmLinkPrev}}" ng-click="prevPage($event)" ng-disabled="page === 1" rel="prev">Prev</a>,
 <a class="button next-page" data-linkname="" ng-attr-data-linkname="{{dtmLinkNext}}" ng-click="nextPage($event)" ng-disabled="page === numberOfPages" rel="next">Next</a>,
 <a class="js-page-1 current-page" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">1</a>,
 <a class="js-page-2" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">2</a>,
 <a class="js-page-3" data-linkname="" ng-attr-data-linkname="{{dtmTracking ? 'pagination-'+($index+1)  : ''}}" ng-class="{'current-page': p === page}" ng-click="goToPage($event, p)">3</a>,
 <a class="js-page-4" data-linkname="" ng-attr-data-linkname=

In [33]:
url3 = "https://www.cars.com/for-sale/searchresults.action/?zc=60606&rd=99999&slrTypeId=28879&yrId=58487&yrId=56007&yrId=51683&yrId=47272&yrId=39723&yrId=34923&yrId=27381&yrId=20201&yrId=20145&yrId=20200&yrId=20144&yrId=20199&yrId=20143&yrId=20198&yrId=20142&yrId=20197&yrId=20141&yrId=20180&stkTypId=28881&sf1Nm=price&sf1Dir=DESC&sf2Nm=miles&sf2Dir=ASC&perPage=50&sortFeatures=buryUsedLowPrice&sortFeatures=buryNewLowPrice&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&searchSource=GN_BREADCRUMB"
page3 = requests.get(url=url3, proxies=proxies)
soup3 = BeautifulSoup(page3.content, 'lxml')
soup3.find_all("a")

[<a class="logo" data-linkname="header-home" href="/" name="&amp;lid=header-home" ng-class="{'simple': smallLogo}" target="_self">
 <svg class="cars-logo" enable-background="new 0 0 165 70" height="70px" version="1.1" viewbox="0 0 165 70" width="165px" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px"><g id="Layer_1"><radialgradient cx="41.1323" cy="-3.0356" gradienttransform="matrix(-0.8303 0.5574 -0.4031 -0.6005 74.0593 -27.7844)" gradientunits="userSpaceOnUse" id="SVGID_1_" r="116.9038"><stop offset="0.3086" style="stop-color:#DEDEDE"></stop><stop offset="0.392" style="stop-color:#9E9E9E"></stop><stop offset="0.522" style="stop-color:#3F3F3F"></stop><stop offset="0.5807" style="stop-color:#1A1A1A"></stop><stop offset="0.6089" style="stop-color:#202020"></stop><stop offset="0.8339" style="stop-color:#4D4D4D"></stop></radialgradient><path d="M15.588,31.615c0-13.049,17.466-24.389,43.138-30.146C24.623,5.791-0.233,19.167-0.